In [12]:
import itertools
from copy import deepcopy
import time
import sys
import random
import pandas as pd
# =================================== SETUP - READING FILES ===================================

def getRules(rulePath):
    if "16" in rulePath:
        c16 = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G']
        rules = []
        for line in open("etc/sudoku-rules-16x16.txt", "r"):
            if line[0] == 'p': continue
            clause = line[:-3].split(" ")
            newclause = []
            for c in clause:
                neg = "-" * (int(c) < 0)            # - sign needed if the predicate is less than 0
                nf = neg + c16[abs(int(c)) // 17**2] + c16[abs(int(c)) % 17**2 // 17] + c16[abs(int(c) % 17**2) % 17]
                newclause.append(nf)
            rules.append(newclause)
        return rules

    else: return [line[:-3].split(' ') for line in open(rulePath, "r")][1:] # Returns the clauses as lists of integers


def getSudokuFromFile(sudokuPath):
    sudokus = [line[:-1] for line in open(sudokuPath, "r")]     # Read in the sudokus from a file
    gridsize = int(len(sudokus[0])**0.5)                        # Dynamically determine grid size based on len(line)
    c16 = ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G']
    cnfs = []
    for sudoku in sudokus:
        clause = []
        for place in range(len(sudoku)):                    # Number format is changed to CNF
            if sudoku[place] != '.':                        # Calculates the place of the number based on the index
                clause.append([c16[place//gridsize] + c16[place % gridsize] + str(sudoku[place])])
        cnfs.append(clause)
    return cnfs

# =================================== DPLL Algorithm ===================================

def dpll(cnf, heuristic):
    global branches, heur_solution, backtrack
    cls = []                                                            # Collects the unit clauses from each iter.
    while True:
        unitClauses = list(set([c[0] for c in cnf if len(c) == 1]))     # Unit clauses are clauses with lenght of 1
        if not unitClauses: break                                       # Quit if there are no more unit clauses
        cls.extend([x for x in unitClauses if x[0] != '-'])                   # Save only the positive predicates
        for p in unitClauses:
            for clause in cnf[:]:
                if p in clause: cnf.remove(clause)                      # Delete the clause with the predicate
                elif neg(p) in clause: cnf[cnf.index(clause)].remove(neg(p))    # Delete the negative of predicate from clause
    heur_solution.extend(list(set(cls)))
    if not cnf:
        solution.extend(list(set(cls)))                 # If the set is empty we satisfied it all
        return list(set(solution))                      # And the unit clauses can be added to the solution
    if not all(cnf):
        heur_solution = list(set(heur_solution) - set(cls))
        backtrack += 1
        return False                       # If a clause is empty it cannot be satisfied
    p = heuristic(cnf)                                  # Choose a heuristic
    branches += 1                                       # Either branch should be correct
    if dpll(deepcopy(cnf) + [[p]], heuristic) or dpll(deepcopy(cnf) + [[neg(p)]], heuristic):
        solution.extend(list(set(cls)))                                 # Add the solution to the set
        return list(set(solution))                                      # Return solutions
    else: return False


def neg(x):
    if x[0] == "-": return x[1:]
    else: return "-" + x

def solver_complete(cnf, heuristic):
    # cnfloop = cnf[:]                                    # ------------- Needs checking for tautologies -------------
    # for clause in cnfloop:                              # Looping trough a copy of the input and
    #     for a, b in itertools.combinations(clause, 2):  # checking if any of the paris in the clauses sum to 0
    #         if (a + b) == 0: cnf.remove(clause)         # ----------------------------------------------------------
    return dpll(cnf, heuristic)                                    # ------- Beginning real algorithm


# =================================== Heuristics ===================================

def heuristics_HUMAN(cnf):
    """
    Looks for clauses with the most filled out rows and columns
    """
    global heur_solution, rules
    #
    size = rules.split('x')[1][0]
    rows = {}
    cols = {}
    for literal in heur_solution:
        if literal[0] in rows:
            rows[literal[0]]+= 1
            if rows[literal[0]] == int(size): del rows[literal[0]]           # Row is full, remove from options
        else: rows[literal[0]] = 1
        if literal[1] in cols:
            cols[literal[1]] += 1
            if cols[literal[1]] == int(size): del cols[literal[1]]  # Row is full, remove from options
        else: cols[literal[1]] = 1

    # Look through the literals and give them a weight based on how filled out their column and row is
    weights = {}
    for clause in cnf:
        for literal in clause:
            if literal not in weights:
                if literal[-3] in rows: w = rows[literal[-3]]
                else: w = 0
                if literal[-2] in cols: w += cols[literal[-2]]
                weights[literal] = w
    return max(weights, key=weights.get)            # Return the most frequent predicate


def heuristics_JWOS(cnf):                               # One-sided Jeroslow-Wang Heuristics
    weights = {}
    for clause in cnf:
        for literal in clause:
            if literal in weights: weights[literal] += 2 ** -len(clause)
            else: weights[literal] = 2 ** -len(clause)
    return max(weights, key=weights.get)                # Return the most frequent predicate


def heuristics_JWTS(cnf):                               # Two-sided Jeroslow-Wang Heuristics
    weights = {}
    for clause in cnf:
        for literal in clause:
            if literal[0] == '-': literal = literal[1:]
            if literal in weights: weights[literal] += 2 ** -len(clause)
            else: weights[literal] = 2 ** -len(clause)
    return max(weights, key=weights.get)                # Return the most frequent predicate


def heuristics_FIRST(cnf):                              # First predicate we encounter
    return cnf[0][0]                                    # Return the first predicate from the first clause


def heuristics_RAND(cnf):                               # Returns a random predicate
    rc = random.randrange(0, len(cnf))                  # Random clause
    rp = random.randrange(0, len(cnf[rc]))                  # Random predicate
    return cnf[rc][rp]                                    # Return the first predicate from the first clause


def heuristics_DLCS(cnf):
    weights = {}
    for clause in cnf:
        for literal in clause:
            if literal in weights:
                weights[literal] += 1
            else:
                weights[literal] = 1
    return max(weights, key=weights.get)            # Return the most frequent predicate


heuristics = {
    'HUMAN' : heuristics_HUMAN,
    'JWOS': heuristics_JWOS,
    'JWTS': heuristics_JWTS,
    'DLCS': heuristics_DLCS,
    'FIRST': heuristics_FIRST,
    'RAND' : heuristics_RAND
}

#  =================================== MAIN PROGRAM ===================================


# --------------- 16 x 16 ---------------
rules_16x16 = "etc/sudoku-rules-16x16.txt"
# sudokuSource = "etc/16x16.txt"

# ---------------- 9 x 9 ----------------
# rules = "etc/sudoku-rules-9x9-rev.txt"
rules_9x9 = "etc/sudoku-rules-9x9.txt"
# sudokuSource = "etc/damnhard.sdk.txt"
sudokuSource = "etc/1000 sudokus.txt"

# ---------------- 4 x 4 ----------------
rules_4x4 = "etc/sudoku-rules-4x4.txt"
# sudokuSource = "etc/4x4.txt"

df = pd.DataFrame(
    columns=["heuristic", "puzzle ID", "time taken", "puzzle", "solution", "backtracking", "branches", "clues given"])

heuristicName = 'FIRST'
heur_solution = []
puzzlefiles = {"etc/1000 sudokus.txt": rules_9x9, "etc/4x4.txt": rules_4x4, "etc/damnhard.sdk.txt": rules_9x9}
tests = 30
rules = rules_9x9
puzzles = getSudokuFromFile("etc/damnhard.sdk.txt")
cnf = getRules(rules_9x9) + puzzles[0]
for heur in heuristics.keys():
    heuristic = heuristics['FIRST']
    #%timeit heuristic(cnf)

In [25]:
%timeit -n 1000 -r 10 heuristics['FIRST'](cnf)

103 ns ± 1.24 ns per loop (mean ± std. dev. of 10 runs, 1000 loops each)


In [26]:
%timeit -n 1000 -r 10 heuristics['RAND'](cnf)

2.38 µs ± 283 ns per loop (mean ± std. dev. of 10 runs, 1000 loops each)


In [27]:
%timeit -n 1000 -r 10  heuristics['HUMAN'](cnf) # Empty heuristic list

2.04 ms ± 446 µs per loop (mean ± std. dev. of 10 runs, 1000 loops each)


In [28]:
%timeit -n 1000 -r 10  heuristics['JWOS'](cnf)

8.66 ms ± 267 µs per loop (mean ± std. dev. of 10 runs, 1000 loops each)


In [29]:
%timeit -n 1000 -r 10  heuristics['JWTS'](cnf)

11.6 ms ± 316 µs per loop (mean ± std. dev. of 10 runs, 1000 loops each)


In [30]:
%timeit -n 1000 -r 10  heuristics['DLCS'](cnf)

3.39 ms ± 216 µs per loop (mean ± std. dev. of 10 runs, 1000 loops each)
